<a href="https://colab.research.google.com/github/Palak-Ai-2006/ML-BOOTCAMP/blob/main/fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split


In [11]:
# Step 1: Download the dataset
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

# Step 2: Unzip the dataset
import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

# Step 3: Define base, train, and validation directories
base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Set up paths for cats and dogs in training and validation directories
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')


--2024-11-14 21:13:03--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.196.207, 74.125.141.207, 74.125.139.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.196.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   212MB/s    in 0.3s    

2024-11-14 21:13:03 (212 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [12]:
from PIL import Image
import numpy as np
import os

# Define paths to the cats and dogs directories
cats_dir = os.path.join(train_dir, "cats")
dogs_dir = os.path.join(train_dir, "dogs")

# Initialize lists to store training data and labels
x_train = []
y_train = []

# Set the number of examples (you can define this as needed)
NUMBER_OF_EXAMPLES = 1000  # Set this to the desired number of examples

# Image dimensions
img_height, img_width = 224, 224

# Loop through the specified number of images
i = 0
while i < NUMBER_OF_EXAMPLES:
    try:
        if i % 2 == 0:  # Load cat images
            im_path = os.path.join(cats_dir, os.listdir(cats_dir)[i])
            im = Image.open(im_path).convert("RGB")
            im_resized = im.resize((img_width, img_height))
            x_train.append(np.array(im_resized))
            y_train.append(1)  # Label for cat
        else:  # Load dog images
            im_path = os.path.join(dogs_dir, os.listdir(dogs_dir)[i])
            im = Image.open(im_path).convert("RGB")
            im_resized = im.resize((img_width, img_height))
            x_train.append(np.array(im_resized))
            y_train.append(0)  # Label for dog

        i += 1
    except IndexError:
        print(f"Not enough images to reach {NUMBER_OF_EXAMPLES}. Ending early.")
        break

# Convert lists to numpy arrays for model training
x_train = np.array(x_train)
y_train = np.array(y_train)

print("Data processing complete.")
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")


Data processing complete.
x_train shape: (1000, 224, 224, 3)
y_train shape: (1000,)


In [17]:
# Load the ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers for binary classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Sigmoid for binary classification

# Final model
model = Model(inputs=base_model.input, outputs=predictions)

In [18]:
# Split data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Convert labels to tensor format for TensorFlow compatibility
y_train = tf.convert_to_tensor(y_train)
y_val = tf.convert_to_tensor(y_val)

In [19]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [20]:
import tensorflow as tf

pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(150, 150, 3),
    pooling='avg',
    classes=2,
    weights='imagenet'
)

In [21]:
history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(x_val, y_val)
)

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.7972 - loss: 0.6556

KeyboardInterrupt: 